In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [17]:
df = pd.read_csv(os.path.join('..', 'mundus', 'data', 'mundus_article_2020.csv'), encoding='ISO-8859-1')
df.sort_values('views', ascending=False, inplace=True)
df.fillna('Divers', inplace=True)
df['count'] = 1
df.head()

,author,date,title,url,views,game,game_type,count
92,Mouchi,2020-09-12 08:22:00,Crusader Kings 3 : Conseils pas si triviaux,https://www.mundusbellicus.fr/593550-crusader-...,8602,Crusader Kings 3,Grande Stratégie,1
189,Mouchi,2020-03-11 22:16:00,"Crusader Kings 3 : Gouvernement, gestion des v...",https://www.mundusbellicus.fr/588131-crusader-...,6294,Crusader Kings 3,Grande Stratégie,1
105,Mouchi,2020-08-11 16:45:00,Crusader Kings 3 : Modding,https://www.mundusbellicus.fr/592628-crusader-...,4216,Crusader Kings 3,Grande Stratégie,1
100,Mouchi,2020-08-26 17:21:00,Crusader Kings 3 : Succès,https://www.mundusbellicus.fr/592955-crusader-...,4139,Crusader Kings 3,Grande Stratégie,1
174,Lord Perhaps On The Roof,2020-04-16 17:15:00,Total War  L'avenir de Warhammer et Three Kin...,https://www.mundusbellicus.fr/589020-total-war...,3237,Divers,Divers,1


In [61]:
len(df)

230

# Jeu

In [60]:
len(df['game'].unique())

35

In [63]:
df2 = df.groupby('game_type').sum()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df2.index, values=df2['count'], textinfo='value+percent'), 1, 1)
fig.add_trace(go.Pie(labels=df2.index, values=df2['views'], textinfo='value+percent'), 1, 2)


fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text='Statistiques 2020 par famille de jeu', separators = ', .',
    annotations=[dict(text='Articles', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Vues', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

# Views by game

In [64]:
fig = px.box(df[df['game_type'] == 'Grande Stratégie'], x='game', y='views',
             title='Vues des articles 2020 de Grande Stratégie')
fig.show()

In [65]:
fig = px.box(df[df['game_type'] == 'Total War'], x='game', y='views',
             title='Vues des articles 2020 de Total War')
fig.show()

In [66]:
fig = px.box(df[df['game_type'] == 'Gestion'], x='game', y='views',
             title='Vues des articles 2020 de Gestion')
fig.show()

# Rédacteurs

In [62]:
len(df['author'].unique())

10

In [58]:
df3 = df.groupby('author').sum()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df3.index, values=df3['count'], textinfo='value+percent'), 1, 1)
fig.add_trace(go.Pie(labels=df3.index, values=df3['views'], textinfo='value+percent'), 1, 2)


fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text='Statistiques 2020 par rédacteur', separators = ', .',
    annotations=[dict(text='Articles', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Vues', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [68]:
fig = px.box(df, x='author', y='views', title='Vues des articles 2020 par rédacteur')
fig.show()